In [2]:
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_circles
from tqdm import tqdm_notebook
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor

In [3]:
import pymorphy2
import re
from nltk.corpus import stopwords

In [4]:
from pymystem3 import Mystem

In [5]:
import nltk
import nltk
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ketmu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
train_df = pd.read_csv("train.csv")

In [7]:
test_df = pd.read_csv("test.csv")

In [8]:
regex = re.compile(u"[А-Яа-яA-Za-z]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

In [9]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также',u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь', 'т', 'д', '-', '-']

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.lower().split() if not token in mystopwords])
    except:
        return u""

In [10]:
train_df["title"] = train_df['title'].apply(words_only)

In [11]:
train_df["title"] = train_df['title'].apply(remove_stopwords)

In [20]:
def print_metrics(y_test, ans):
    print(("Accuracy ~"), np.round(accuracy_score(y_test, ans), 4))
    print(("F1_score ~"), np.round(f1_score(y_test, ans, average="macro"), 4))

In [13]:
vectorizer = TfidfVectorizer()

In [14]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values
X_train_vectorized = vectorizer.fit_transform(X_train)

In [15]:
train_df.head()

,id,url,title,target
0,0,m.kp.md,экс министр экономики молдовы главе мидэи цель...,False
1,1,www.kp.by,эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,банши сезон серия бремя красоты смотреть онлайн,False
3,3,colorbox.spb.ru,беси картинки,False
4,4,tula-sport.ru,новомосковске сыграют следж хоккеисты алексинс...,False


In [16]:
RF2 = RandomForestClassifier(n_estimators=2000, min_samples_leaf=1, min_samples_split=6, n_jobs=-1)#best

In [17]:
%%time
RF2.fit(X_train_vectorized, y_train.ravel())

Wall time: 1h 13min 31s


RandomForestClassifier(min_samples_split=6, n_estimators=2000, n_jobs=-1)

In [18]:
RF2_ans = RF2.predict(X_train_vectorized)

In [21]:
print_metrics(y_train, RF2_ans)

Accuracy ~ 0.998
F1_score ~ 0.9954


In [22]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = RF2.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)